<a href="https://colab.research.google.com/github/Anchaliya75/Convolve/blob/main/Convolve_PS2_Oversampling_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import numpy as np

In [54]:
df = pd.read_excel('/content/FileAndResults_Problem2.xlsx')

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   File Changed   351 non-null    object
 1   Component      351 non-null    object
 2   Coder          351 non-null    object
 3   Tester         337 non-null    object
 4   Category       351 non-null    object
 5   Sanity name    351 non-null    object
 6   nets Tested    351 non-null    object
 7   Sanity Result  351 non-null    object
dtypes: object(8)
memory usage: 22.1+ KB


In [56]:
df.describe().transpose()

count unique                     top freq
File Changed    351    176   cpp/csyu/category/fnf    2
Component       351     45       asr1001-forge-cgn   70
Coder           351      7                 patrick  130
Tester          337     18             manishakang  109
Category        351     11              PD-asr1001  146
Sanity name     351     25              vistra-cxr  122
nets Tested     351     14  asr1001-PX,asr1001-X64  105
Sanity Result   351      3                    Pass  210

In [57]:
new_df = pd.get_dummies(df,columns=['Coder','Tester','Component','Category','Sanity name','nets Tested'])

In [58]:
new_df.drop(columns='File Changed',inplace =True)

In [59]:
new_df.head()

Sanity Result  Coder_Alex  Coder_Batukbaj  Coder_abhimanyu  \
0          Pass           0               1                0   
1       Aborted           1               0                0   
2          Fail           1               0                0   
3       Aborted           1               0                0   
4       Aborted           1               0                0   

   Coder_actalivineo  Coder_chris  Coder_daffoil  Coder_patrick  \
0                  0            0              0              0   
1                  0            0              0              0   
2                  0            0              0              0   
3                  0            0              0              0   
4                  0            0              0              0   

   Tester_akafsg  Tester_arvenugo  ...  nets Tested_NCS560,SPITFIRE,NCS540L  \
0              0                0  ...                                    0   
1              0                0  ...                                    0   
2              0                0  ...                                    0   
3              0                0  ...                                    0   
4              0                0  ...                                    0   

   nets Tested_NCS5K,NCS560  nets Tested_NCS6K  nets Tested_SPITFIRE,NCS4K  \
0                         0                  0                           0   
1                         0                  0                           0   
2                         0                  0                           0   
3                         0                  0                           0   
4                         0                  0                           0   

   nets Tested_SPITFIRE,NCS540L  nets Tested_XRV9K  \
0                             0                  0   
1                             0                  0   
2                             0                  0   
3                             0                  0   
4                             0                  0   

   nets Tested_asr1001-PX,asr1001-X64  nets Tested_asr1001-X64  \
0                                   0                        0   
1                                   0                        0   
2                                   0                        0   
3                                   0                        0   
4                                   0                        0   

   nets Tested_asr1001-X64,asr1001-PX  nets Tested_saraja-PX  
0                                   0                      0  
1                                   0                      0  
2                                   0                      0  
3                                   0                      0  
4                                   0                      0  

[5 rows x 121 columns]

In [60]:
Label = new_df.loc[:,['Sanity Result']]

In [61]:
Label.head()

Sanity Result
0          Pass
1       Aborted
2          Fail
3       Aborted
4       Aborted

In [62]:
Label.replace('Pass', 2, inplace = True)
Label.replace('Aborted', 1,  inplace = True) 
Label.replace('Fail', 0,  inplace = True)

In [63]:
Label.head()

Sanity Result
0              2
1              1
2              0
3              1
4              1

In [64]:
X= new_df.drop(columns = ['Sanity Result'])
y = Label

In [65]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(n_components=2)
lda.fit(X,y)
X_t=lda.transform(X)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [66]:
X_t =pd.DataFrame(X_t,columns=['A','B'])

In [67]:
from imblearn.over_sampling import RandomOverSampler

ros =  RandomOverSampler()
X_ros, y_ros = ros.fit_resample(X_t, y)

In [68]:
y_ros['Sanity Result'].unique()

array([2, 1, 0])

In [69]:
y_ros[y_ros['Sanity Result']==2]

Sanity Result
0                2
8                2
9                2
10               2
11               2
..             ...
338              2
342              2
343              2
346              2
348              2

[210 rows x 1 columns]

In [70]:
y_ros[y_ros['Sanity Result']==1]

Sanity Result
1                1
3                1
4                1
5                1
6                1
..             ...
625              1
626              1
627              1
628              1
629              1

[210 rows x 1 columns]

In [71]:
y_ros[y_ros['Sanity Result']==0]

Sanity Result
2                0
7                0
20               0
29               0
32               0
..             ...
485              0
486              0
487              0
488              0
489              0

[210 rows x 1 columns]

In [72]:
from sklearn.model_selection import StratifiedShuffleSplit 

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=0)

for train,test in sss.split(X_ros,y_ros):    
    X_train_SS = X_ros.iloc[train]
    y_train_SS = y_ros.iloc[train]
    X_test_SS = X_ros.iloc[test]
    y_test_SS = y_ros.iloc[test]

In [73]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [74]:
from sklearn.linear_model import LogisticRegression,RidgeClassifier, Perceptron
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

classifiers = {
    "Logistic Regression": LogisticRegression(class_weight='balanced'),
    "Decision Tree": DecisionTreeClassifier(class_weight='balanced'),
    "k-Nearest Neighbors": KNeighborsClassifier(),
    "Linear SVM": SVC(class_weight='balanced'),
    "Gradient Boosting Classifier": GradientBoostingClassifier(),
    "Random Forest": RandomForestClassifier(),
    'RidgeClassifier': RidgeClassifier(class_weight='balanced'),
    'AdaBoost': AdaBoostClassifier(n_estimators=100),
    # 'MNB': MultinomialNB(),
    'Perceptron': Perceptron(class_weight='balanced'),
    'xgboost': XGBClassifier(n_estimators=300),
    'catboost': CatBoostClassifier(verbose=0)
        }

In [75]:
no_classifiers = len(classifiers.keys())

from sklearn.metrics import f1_score
from time import process_time 

def batch_classify(X_train_tranformed, y_train, X_test_tranformed, y_test, verbose = True):
    df_results = pd.DataFrame(data=np.zeros(shape=(no_classifiers,3)), columns = ['Classifier', 'f1', 'Training time'])
    count = 0
    for key, classifier in classifiers.items():
        t_start = process_time()  
        classifier.fit(X_train_tranformed, y_train)
        t_stop = process_time() 
        t_elapsed = t_stop - t_start
        y_predicted = classifier.predict(X_test_tranformed)
        
        df_results.loc[count,'Classifier'] = key
        df_results.loc[count,'f1'] = f1_score(y_test, y_predicted, average = 'macro')
        df_results.loc[count,'Training time'] = t_elapsed
        if verbose:
            print("trained {c} in {f:.2f} s".format(c=key, f=t_elapsed))
        count+=1

    return df_results

In [76]:
df_results = batch_classify(X_train_SS, y_train_SS,X_test_SS, y_test_SS)
print(df_results.sort_values(by='f1', ascending=False))

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ra

trained Logistic Regression in 0.01 s
trained Decision Tree in 0.00 s
trained k-Nearest Neighbors in 0.00 s
trained Linear SVM in 0.00 s
trained Gradient Boosting Classifier in 0.29 s
trained Random Forest in 0.14 s
trained RidgeClassifier in 0.01 s


<ipython-input-75-3c1466a05748>:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_tranformed, y_train)
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:1059: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


trained AdaBoost in 0.17 s
trained Perceptron in 0.01 s
trained xgboost in 0.11 s


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


trained catboost in 1.48 s
                      Classifier        f1  Training time
1                  Decision Tree  0.851758       0.003963
9                        xgboost  0.849546       0.105928
5                  Random Forest  0.844163       0.141230
4   Gradient Boosting Classifier  0.840373       0.285006
10                      catboost  0.840373       1.478898
8                     Perceptron  0.837462       0.005878
6                RidgeClassifier  0.837109       0.006606
0            Logistic Regression  0.834263       0.014554
3                     Linear SVM  0.833221       0.004940
2            k-Nearest Neighbors  0.824414       0.003721
7                       AdaBoost  0.775443       0.167706
